# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector
import json
from tqdm import tqdm

# Import data

In [ ]:
# Read offers_canarias.json as a DataFrame in the folder data
df = pd.read_json('data/offers_canarias.json', encoding='utf-8')
# Add the column 'salaryOriginal' with NaN values
df['salaryOriginal'] = np.nan
# Define a custom function to access the 'label' key in a dictionary
def get_label(d):
    return d.get('label')
# Apply the custom function to the 'location' and 'category' columns
df['location'] = df['location'].apply(get_label)
df['category'] = df['category'].apply(get_label)
print(df.shape)
df.head()

# Test connection to database

In [ ]:
# Test the connection to the database
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'test',
    'port': 33060
}

# SELECT * FROM test.jobmarket_canarias;
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

# Declare functions

In [ ]:
def execute_query(query):
    # Read .sql file in sql folder
    with open('sql/' + query + '.sql', 'r') as file:
        query = file.read()
    # Execute query
    cursor.execute(query)
    # Commit changes
    cnx.commit()

def get_data_from_query(query):
    # Read .sql file in sql folder
    with open('sql/' + query + '.sql', 'r') as file:
        query = file.read()
    # Execute query
    cursor.execute(query)
    # Fetch data
    data = cursor.fetchall()
    # Return data
    return data

# Create table

# Describe table

In [ ]:
# Describe the table
cursor.execute('DESCRIBE test.ofertas_jobmarket_canarias_21_23')
# Print the result
describe = cursor.fetchall()
# Print the column names
columns = [column[0] for column in describe]

# Get salary

In [ ]:
def get_salary_to_df(df):
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        postings = row['postings']
        for posting in postings:
            if 'salaryOriginal' in posting:
                df.loc[index, 'salaryOriginal'] = posting['salaryOriginal']
    return df

df = get_salary_to_df(df)

# Delete tables of database

In [ ]:
def delete_table(cursor, table_name):
    cursor.execute("DROP TABLE IF EXISTS {}".format(table_name))
    print("Finished deleting table (if existed).")

def create_table(cursor, table_name, columns):
    cursor.execute("CREATE TABLE {} ({})".format(table_name, columns))
    print("Finished creating table.")
    
# delete_table(cursor, 'test.ofertas_jobmarket_canarias_21_23')
# create_table(cursor, 'test.ofertas_jobmarket_canarias_21_23', describe)

# Import data to database

In [ ]:
def import_data_to_db(df):
    # Iterate over the rows in the DataFrame
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        # Create an object with all the columns except "postings"
        row_data = row.drop('postings')
        row_data = row_data.to_dict()
        # Replace NaN values with None
        for key, value in row_data.items():
            if pd.isna(value):
                row_data[key] = None
        # Serialize the dictionary to a JSON string
        row_data_json = json.dumps(row_data)
        # Insert into the ofertas_jobmarket_canarias_21_23 table and get the id
        try:
            cursor.execute('INSERT INTO test.ofertas_jobmarket_canarias_21_23 ({}) VALUES ({})'.format(', '.join(row_data.keys()), ', '.join(['%s'] * len(row_data))), list(row_data.values()))
        except mysql.connector.errors.DataError:
            continue
        id = cursor.lastrowid
        cursor.commit()
        # Add id to postings
        row['id'] = id
    return df
df = import_data_to_db(df)

In [ ]:
df.head()

In [ ]:
def import_postings_to_db(df):
    # Iterate over the postings_data list with a tqdm progress bar
    for posting in tqdm(postings_data, desc="Inserting postings"):
        try:
            # Flatten the nested dictionaries within the posting dictionary
            flattened_posting = {
                'date': posting['date'],
                'site_label': posting['site']['label'],
                'site_key': posting['site']['key'],
                'titleOriginal': posting['titleOriginal'],
                'url': posting['url'],
                'id_posting': posting['id_posting'],
                'id': posting['id']
            }
            # Convert the dictionary values into a list of values
            values = list(flattened_posting.values())
            # Generate placeholders for the SQL query
            placeholders = ', '.join(['%s'] * len(values))
            # Construct and execute the SQL query
            cursor.execute(f'INSERT INTO test.postings_jobmarket_canarias_21_23 ({columns}) VALUES ({placeholders})', values)
        except mysql.connector.errors.DataError:
            continue
        cnx.commit()
# import_postings_to_db(df)

# Test MySQL table

In [ ]:
import time

def timer(function=None, *args, **kwargs):
    start = time.time()
    data = function(*args, **kwargs)
    end = time.time()
    print(f"Time elapsed: {end - start}")

def get_offers():
    cursor.execute('SELECT * FROM test.ofertas_jobmarket_canarias_21_23')
    data = cursor.fetchall()
    return data

def get_postings():
    cursor.execute('SELECT * FROM test.postings_jobmarket_canarias_21_23')
    data = cursor.fetchall()
    return data

def get_offers_and_postings():
    query = """
    SELECT * FROM test.ofertas_jobmarket_canarias_21_23 AS o
    INNER JOIN test.postings_jobmarket_canarias_21_23 AS p
    ON o.id = p.id
    """
    cursor.execute(query)
    data = cursor.fetchall()
    return data

def get_offer_by_id(id):
    query = f"SELECT * FROM test.ofertas_jobmarket_canarias_21_23 WHERE id = {id}"
    cursor.execute(query)
    data = cursor.fetchall()
    return data

def get_offers_by_dates(start_date, end_date):
    query = f"""
    SELECT * FROM test.ofertas_jobmarket_canarias_21_23
    WHERE fecha_publicacion BETWEEN '{start_date}' AND '{end_date}'
    """
    cursor.execute(query)
    data = cursor.fetchall()
    return data

def get_offer_by_category_and_period(start_date, end_date,category):
    query = f"""
    SELECT categoria, count(*) FROM test.ofertas_jobmarket_canarias_21_23
    WHERE fecha_publicacion BETWEEN '{start_date}' AND '{end_date}'
    GROUP BY categoria
    ORDER BY 2 DESC
    """
    cursor.execute(query)
    data = cursor.fetchall()
    return data

timer(get_offers)

timer(get_postings)

timer(get_offers_and_postings)

timer(get_offer_by_id, 1)